In [11]:
#import necessary libraries 

import pandas as pd
import streamlit as st
import plotly.express as px


#download and look at vehicle data to determine what to do next
vehicle_data = pd.read_csv('vehicles_us.csv')
print(vehicle_data.info())

#check for obvious duplicates
duplicate_vehicles = vehicle_data.duplicated().sum()
print(duplicate_vehicles)

#check for missing values
missing_vehicles = vehicle_data.isnull().sum()
display(missing_vehicles)

#create a function that replaces the NaNs with the median value and apply to each required column
def median_value(group):
    return group.fillna(group.median())

vehicle_data['model_year'] = vehicle_data.groupby('model')['model_year'].transform(median_value)
vehicle_data['cylinders'] = vehicle_data.groupby('model')['cylinders'].transform(median_value)
vehicle_data['odometer'] = vehicle_data.groupby('model')['odometer'].transform(median_value)


display(vehicle_data.sample(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB
None
0


price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

c:\Users\Zachary\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
708,25400,2015.0,ram 2500,excellent,8.0,diesel,178285.0,automatic,truck,white,NaN,11/24/2018,13
19390,5995,2008.0,buick enclave,excellent,6.0,gas,170000.0,automatic,SUV,black,NaN,2/11/2019,18
33227,28990,2015.0,ford f-350 sd,good,8.0,diesel,112150.0,automatic,truck,white,1.0,4/14/2019,71
23663,5900,2009.0,toyota corolla,good,4.0,gas,33235.0,automatic,sedan,grey,NaN,6/10/2018,13
6871,12995,2007.0,chevrolet tahoe,good,8.0,gas,145129.0,automatic,SUV,silver,NaN,1/12/2019,30
10517,5500,2011.0,hyundai sonata,excellent,4.0,gas,85000.0,automatic,sedan,silver,NaN,10/9/2018,22
896,7250,2010.0,dodge grand caravan,excellent,6.0,gas,109298.0,automatic,mini-van,black,NaN,11/17/2018,12
22535,3500,2003.0,chevrolet silverado,good,8.0,gas,204000.0,automatic,pickup,white,1.0,3/22/2019,67
11376,18000,2018.0,chevrolet malibu,like new,4.0,gas,3063.0,automatic,sedan,white,NaN,11/7/2018,43
26902,12500,2011.0,chevrolet silverado 1500,excellent,8.0,gas,120000.0,automatic,truck,white,NaN,2/7/2019,14
